In [5]:
import os
import numpy as np
import pylab
import imageio
from matplotlib import pyplot as plt
import cv2
import time
from os.path import isfile, join
from keras.applications import mobilenet
from keras.models import load_model
from scipy.ndimage.measurements import label
from scipy.ndimage.measurements import center_of_mass
from matplotlib import colors
import skimage
from keras.preprocessing.image import ImageDataGenerator

In [6]:
print(os.listdir('.'))

['.ipynb_checkpoints', 'alt.csv', 'Bebop', 'Bebop2_20180414154341-0700.mp4', 'Bebop2_20180414163256-0700.mp4', 'bebop_mobilenet_v0.h5', 'Frames', 'Frames_New', 'out.mp4', 'out1.mp4', 'Semantic_Segmentation.ipynb', 'Video Frame Extraction.ipynb', 'Working CNN Boi.ipynb']


In [7]:
# normalization
# normalize each chip
samplewise_center = True
samplewise_std_normalization = True
# normalize by larger batches
featurewise_center = False
featurewise_std_normalization = False

# adjacent pixel correllation reduction
# never explored
zca_whitening = False
zca_epsilon = 1e-6

# data augmentation 
# training only
transform = 0
zoom_range = 0
color_shift = 0
rotate = 0
flip = False

datagen_test = ImageDataGenerator(
        samplewise_center=samplewise_center,
        featurewise_center=featurewise_center,
        featurewise_std_normalization=featurewise_std_normalization,
        samplewise_std_normalization=samplewise_std_normalization,
        zca_whitening=zca_whitening,
        zca_epsilon=zca_epsilon,
        rotation_range=rotate,
        width_shift_range=transform,
        height_shift_range=transform,
        shear_range=transform,
        zoom_range=zoom_range,
        channel_shift_range=color_shift, 
        fill_mode='constant',
        cval=0,
        horizontal_flip=flip,
        vertical_flip=flip,
        rescale=1./255,
        preprocessing_function=None)


**Module to operate on each individual frame of the video**

In [8]:
#Load Weights
model = load_model('bebop_mobilenet_v0.h5', custom_objects={
                   'relu6': mobilenet.relu6,
                   'DepthwiseConv2D': mobilenet.DepthwiseConv2D})

In [9]:
def ProcessChip (frame):
    #result_feature_map = np.zeros((9,16,7))  #CNN feature map to be returned
    values = np.zeros((9,16,3))
    chips = np.zeros((144,120,120,3))
    
    for i in range(0,9):
        for j in range(0,16):
            chips[16*i+j] = frame[120*i:120*(i+1), 120*j:120*(j+1), :]
            
    generator_test = datagen_test.flow(
        chips, 
        batch_size=144,
        shuffle=False)
    
    #return values
    return model.predict_generator(generator_test,
                                  steps = 1)

In [10]:
#All Decision Algo Definition

#Function to find the closest roof/driveway
def closest(list,img_center):
    closest=list[0]
    for c in list:
        if np.linalg.norm(c-img_center) < np.linalg.norm(closest-img_center):
            closest = c
    return closest

#Sliding window function
def sliding_window_view(arr, shape):
    n = np.array(arr.shape) 
    o = n - shape + 1 # output shape
    strides = arr.strides
    
    new_shape = np.concatenate((o, shape), axis=0)
    new_strides = np.concatenate((strides, strides), axis=0)
    return np.lib.stride_tricks.as_strided(arr ,new_shape, new_strides)

In [11]:
##Decision algo with input of 9x16 array at which image was taken.
def decision_algo(image_frame):
    image_frame[image_frame==0]=3
    
    ### READ THE ALTITUDE FROM CSV FILE ###
    #Read alt.csv
    with open('alt.csv', 'r') as csvfile:
        alt_list = [line.rstrip('\n') for line in csvfile]

    #Choose last value in alt_list        
    altitude=int(alt_list[-1]) #in meters

    
    ### ALGORITHM TO FIND CLOSEST DRIVEWAY ###
    #Center of the 9x16 array
    img_center=np.array([4,7.5])

    #Label all the driveways and roofs
    driveway, num_driveway = label(image_frame==1)
    roof, num_roof = label(image_frame==2)

    #Save number of driveways and roofs into array
    d=np.arange(1,num_driveway+1)
    r=np.arange(1,num_roof+1)
    
    if(len(d)<1):
        print("No driveway found, return to base")
    else:
        #Find the center of the all the driveways
        driveway_center=center_of_mass(image_frame,driveway,d)
        roof_center=center_of_mass(image_frame,roof,r)

        #Find the closest roof to the center of the image
        if(len(roof_center)>0):
            closest_roof=closest(roof_center,img_center)
        else:
            #if no roof is found, set closest_roof as center of image
            closest_roof=img_center
            print("Roof center list empty")

        #Find the closest driveway to the closest roof
        closest_driveway=closest(driveway_center,np.asarray(closest_roof))

        ### ALGORITHM TO FIND 3x3 DRIVEWAY TO LAND ###
        #If altitude is 5m or less, look for a 3x3 sliding window of 1's, if found, Land.
        #At 5m, a 3x3 will be equivalent to 1.5m x 1.5m.
        if(altitude<=5.0):
            #Creates a 7x10 ndarray with all the 3x3 submatrices
            sub_image=sliding_window_view(image_frame,(3,3))

            #Empty list
            driveway_list=[]

            #Loop through the 7x14 ndarray
            for i in range(0,7):
                for j in range(i,14):
                    #Calculate the total of the  submatrices
                    output=sum(sum(sub_image[i,j]))
                    #if the output is 9, that means we have a 3x3 that is all driveway
                    if output==9:
                        #append the i(row) and j(column) to a list declared previously
                        #we add 1 to the i and j to find the center of the 3x3
                        driveway_list.append((i+1,j+1))

            if(len(driveway_list)>0):
                #Call closest function to find driveway closest to house.             
                closest_driveway=closest(driveway_list,np.asarray(closest_roof))
                print(closest_driveway)
                print("Safe to land")
            else:
                print("Need to fly lower")
        


        ### SCALE CLOSEST DRIVEWAY CENTER TO REAL WORLD COORDINATES AND SAVE TO CSV ###
        scaler=0.205/(216.26*altitude**-0.953) #m/pixel
        if(len(driveway_center)>0):
            print (closest_driveway)
            move_coordinates=([4,7.5]-np.asarray(closest_driveway)) #Find coordinates relative to center of image
            move_coordinates=np.asarray(move_coordinates)*np.asarray(scaler)*120 #60 is the center of the 120x120 superpixel
            move_coordinates=np.append(move_coordinates,(altitude-2)) #Add altitude to array
            print (move_coordinates)
            with open('coords.csv', 'w') as csvfile:
                filewriter = csv.writer(csvfile, delimiter=',')
                filewriter.writerow(move_coordinates)
            with open('coordinates_history.csv', 'a', newline='') as csvfile:
                filewriter = csv.writer(csvfile, delimiter=',')
                filewriter.writerow(move_coordinates)      

    return

In [40]:
def heatmap (feature_map, frame):
    color_mask = np.zeros((1080,1920,3))
    temp_frame = skimage.img_as_float(frame)
    alpha = 0.6
    for i in range (0,9):
        for j in range (0,16):
            if feature_map[i][j] == 2:
                color_mask[120*i:120*(i+1), 120*j:120*(j+1), :] = [0, 0, 1] #Blue, House
            elif feature_map[i][j] == 1:
                color_mask[120*i:120*(i+1), 120*j:120*(j+1), :] = [0, 1, 0] #Green, Concrete
            else:
                color_mask[120*i:120*(i+1), 120*j:120*(j+1), :] = [1, 0, 0] #Red, Don't Care
    color_mask_hsv = colors.rgb_to_hsv(color_mask)
    frame_hsv = colors.rgb_to_hsv(temp_frame)
    frame_hsv[..., 0] = color_mask_hsv[..., 0]
    frame_hsv[..., 1] = color_mask_hsv[..., 1] * alpha
    frame_masked = colors.hsv_to_rgb(frame_hsv)
    return frame_masked

In [13]:
def correct_arr (arr) :
    arr = arr + 1
    arr[arr>2] = 0
    return arr

**Module to iterate through each frame in video**

In [30]:
def VideoToFrames (vid):
    
    count = 0 # Can be removed. Just to verify number of frames
    #count_pavement = []
    t = time.time()
    for image in vid.iter_data(): #Iterate through every frame in Video
        #image: numpy array containing image information
        
        if count % 100 == 0:
            feature_map = ProcessChip(image)
            arr = heatmap(np.reshape(correct_arr(np.argmax(ProcessChip(image), axis=1)), (9,16)), image)
            cv2.imwrite('./Frames_New//frame%d.jpg'%count, arr*255)
        count += 1
    elapsed = time.time() - t 
    return elapsed

In [15]:
def convert_frames_to_video(pathIn,pathOut,fps):
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
 
    #for sorting the file names properly
    files.sort(key = lambda x: int(x[5:-4]))
 
    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each file
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        print(filename)
        #inserting the frames into an image array
        frame_array.append(img)
 
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
 
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()

In [35]:
filename = './Bebop/Bebop2_20180422173922-0700.mp4' #Add path to video file

In [36]:
vid = imageio.get_reader(filename, 'ffmpeg') #You can use any reader of your choice
#print (vid.iter_data())

In [37]:
time_taken = VideoToFrames(vid) #Passing the video to be analyzed frame by frame

In [38]:
print ('Total time taken %s'%time_taken)

Total time taken 270.3919630050659


In [41]:
convert_frames_to_video('./Frames_New/', 'out1.mp4', 2.5)

./Frames_New/frame0.jpg
./Frames_New/frame1.jpg
./Frames_New/frame100.jpg
./Frames_New/frame200.jpg
./Frames_New/frame300.jpg
./Frames_New/frame400.jpg
./Frames_New/frame500.jpg
./Frames_New/frame600.jpg
./Frames_New/frame700.jpg
./Frames_New/frame800.jpg
./Frames_New/frame900.jpg
./Frames_New/frame1000.jpg
./Frames_New/frame1100.jpg
./Frames_New/frame1200.jpg
./Frames_New/frame1300.jpg
./Frames_New/frame1400.jpg
./Frames_New/frame1500.jpg
./Frames_New/frame1600.jpg
./Frames_New/frame1700.jpg
./Frames_New/frame1800.jpg
./Frames_New/frame1900.jpg
./Frames_New/frame2000.jpg
./Frames_New/frame2100.jpg
./Frames_New/frame2200.jpg
./Frames_New/frame2300.jpg
./Frames_New/frame2400.jpg
./Frames_New/frame2500.jpg
./Frames_New/frame2600.jpg
./Frames_New/frame2700.jpg
./Frames_New/frame2800.jpg
./Frames_New/frame2900.jpg
./Frames_New/frame3000.jpg
./Frames_New/frame3100.jpg
./Frames_New/frame3200.jpg
./Frames_New/frame3300.jpg
./Frames_New/frame3400.jpg
./Frames_New/frame3500.jpg
./Frames_New/fra